![Callysto.ca Banner](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-top.jpg?raw=true)

<a href="https://hub.callysto.ca/jupyter/hub/user-redirect/git-pull?repo=https%3A%2F%2Fgithub.com%2Fcallysto%2Flesson-plans&branch=master&subPath=notebooks/3Dprinting/3Dprinting-polar.ipynb&depth=1"><img src="https://raw.githubusercontent.com/callysto/curriculum-notebooks/master/open-in-callysto-button.svg?sanitize=true" width="123" height="24" alt="Open in Callysto"/></a>

## 3D Printing in Polar Coordinates

This short notebook demonstrates how to plot a surface in 3D and convert it into an STL file suitable for printing on any standard 3D printer. 

This is a modification of an earlier notebook, 3Dprinting.ipynb. Here, we use polar coordinates instead of Cartesian coordinates. 

There are two sections to the notebook
- create an STL file for a sphere, defined by a mathematical formula in polar coordinates
- view the files, and print on a 3D printer

We start by installing the numpy-stl library, then importing it and a few other necessary libraries. 

In [ ]:
!pip install numpy-stl

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from stl import mesh

## The  example. A sphere

Here we demonstrate how to print a surface given from a mathematical function, described using polar coordinates.

Let's make a visual plot of a sphere, given by the formula  $$x^2 + y^2 + z^2 = 1. $$

We can plot the top part of the sphere by solving for $z$, with $$ z = \sqrt{1 - x^2 - y^2}.$$

For polar form, take rectangular grid in the r-$\theta$ polar coordinates in the  plane, create a meshgrid, and then define indexed arranges $X,Y,Z$ using the formulas for polar coordinates. These formulas for $x,y,z$ are

- $ x = r\cos(\theta)$
- $ y = r\sin(\theta)$
- $ z = \sqrt{1-r^2} $ 

Here is the code and the plot:

In [ ]:
# Code for a 3D surface plot

# set the number of points in r and theta directions
Nr = 10
Nt = 10

r = np.linspace(0,1,Nr)
t = np.linspace(0,1,Nt)

# create a mesh grid in x and y, then define z values.
R, T = np.meshgrid(r, t)
X = R*np.cos(2*np.pi*T)
Y = R*np.sin(2*np.pi*T)
Z = np.sqrt(1-R*R)

# now we plot
fig = plt.figure(figsize = (10,10))
ax = plt.axes(projection='3d')
psurf = ax.plot_surface(X, Y, Z)
ax.set_xlabel('x', labelpad=20)
ax.set_ylabel('y', labelpad=20)
ax.set_zlabel('z', labelpad=20)

plt.show()

## Triangulating the surface.

To triangulate the surface, we break up each cell in the grid as a union of two triangles, oriented counterclockwise. 

<div align="center">
<img src="images/Cell_triangles.png" alt="A grid cell with two triangles" width="400"/><br>
A grid cell with two oriented triangles.
</div>

The coordinates of the three vertices in each triangle are fed into the **vectors** dataset of the 3D print mesh. Noting the coordinates for x,y in the diagram above, the three vertices of the red triangle are given as

```
bottom left: [ X[i,j],   Y[i,j],     Z[i,j] ]
  top right: [ X[i+1,j+1],Y[i+1,j+1],Z[i+1,j+1] ]
   top left: [ X[i+1,j],  Y[i+1,j],  Z[i+1,j] ],
```
while the coordinates for the blue triangle are given as
```
 bottom left: [ X[i,j],    Y[i,j],    Z[i,j] ]
bottom right: [ X[i,j+1],  Y[i,j+1],  Z[i,j+1] ]
   top right: [ X[i+1,j+1],Y[i+1,j+1],Z[i+1,j+1] ].
```

The 3D printer expects to see a bottom surface as well, so we get set $Z0 = -Z $ to get a bottom half of the sphere. 
This second surface also must be triangulated and vectors added to the mesh. Since it is on the bottom, the oriented triangles go in the opposite directions 

Unlike the earlier notebook, there are no edges needed to connect these two surfaces. So we can skip that part of the code. 

Finally, we write the code to create the surface mesh that is used to create the STL file for printing. Note that the arrays $X,Y,Z$ define points on our vertices and we don't need to explicitly label the faces. This makes the code easier to understand, as we just loop over grid points to create the final data structure. 

The two surfaces have $Nr*Nt$ points each, giving $(Nr-1)*(Nt-1)$ rectangles each, and so we have $4(Nx-1)*(Ny-1)$ triangles for their triangulation. The total number of triangles (faces) is
$$\mbox{Nfaces } = 4(Nx-1)(Ny-1).$$

Here is the code:

In [ ]:
Z0 = -Z  # The bottom surface, the negative ot the top

Nfaces = 4*(Nr-1)*(Nt-1)
surf = mesh.Mesh(np.zeros(Nfaces, dtype=mesh.Mesh.dtype))
k = 0

# top surface grid, two triangles per rectangular cell
for i in range(Nt-1):
    for j in range(Nr-1):
        surf.vectors[k][0] = [ X[i,j],Y[i,j],Z[i,j] ]
        surf.vectors[k][1] = [ X[i+1,j+1],Y[i+1,j+1],Z[i+1,j+1] ]
        surf.vectors[k][2] = [ X[i+1,j],Y[i+1,j],Z[i+1,j] ]
        k +=1
        surf.vectors[k][0] = [ X[i,j],Y[i,j],Z[i,j] ]
        surf.vectors[k][1] = [ X[i,j+1],Y[i,j+1],Z[i,j+1] ]
        surf.vectors[k][2] = [ X[i+1,j+1],Y[i+1,j+1],Z[i+1,j+1] ]
        k += 1
# bottom surface grid, two triangles per rectangular cell. Note clockwise order!
for i in range(Nt-1):
    for j in range(Nr-1):
        surf.vectors[k][0] = [ X[i,j],Y[i,j],Z0[i,j] ]
        surf.vectors[k][2] = [ X[i+1,j+1],Y[i+1,j+1],Z0[i+1,j+1] ]
        surf.vectors[k][1] = [ X[i+1,j],Y[i+1,j],Z0[i+1,j] ]
        k +=1
        surf.vectors[k][0] = [ X[i,j],Y[i,j],Z0[i,j] ]
        surf.vectors[k][2] = [ X[i,j+1],Y[i,j+1],Z0[i,j+1] ]
        surf.vectors[k][1] = [ X[i+1,j+1],Y[i+1,j+1],Z0[i+1,j+1] ]
        k += 1



# Write the mesh to file "sphere.stl"
surf.save('sphere.stl')

## Viewing your results

You now have a file in your Jupyter hub called sphere.stl that you can view and print. Download this file to your local computer, then drag and drop into an online STL viewer such as https://www.viewstl.com/


## 3D Printing the file - final steps

You now have a file in your Jupyter hub called sphere.stl that you can print. Download this file to your local computer, then open it with your favourite 3D printing software. For instance, I am using PrusaSlicer, with is based on Slic3r. Use the Slicer software to adjust the size, add supports if necessary and adjust any other settings for the filament type and quality of print you require. Then create a g-code file and transfer this to your 3D printer. 

And print!  

Here is a sample result, using PLA filament in draft mode on a Prusa printer. The support columns at the base are still attached. 

<div align="center">
<img src="images/Sphere_with_supports.jpg" alt="The final printed sphere" width="400"/><br>
The final printed sphere
</div>

## Going further

We can try several ideas to explore this 3D printing:

- try printing the surface with fewer supports on the base. It is symmetric so it may print well without supports.
- increase the number of grid points to obtain a smoother printed surface
- use a different function to get a different surface
  - eg, a paraboloid is given by the equation $z = x^2 + y^2 = r^2$
  - a cone is given by $z = r$, inverted cone by $z = 1-r$
  - a wavey function is given by $z = \sin(10*(r^2))$

Experiment and print.

[![Callysto.ca License](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-bottom.jpg?raw=true)](https://github.com/callysto/curriculum-notebooks/blob/master/LICENSE.md)